In [1]:
from Search.resultsConnector import Connector
from Search.msdialTxtReader import msdialReader
from Search.mgfReader import mgfReader
from Search.searchSterol import searchMSDIALAlignExportWithMGF
from utils.utils import getfilepath
import pandas as pd

In [2]:
#--------------------convert mgf to csv ----------------------------------
mgfFolder = "./Search/tissue/kidney" # input a folder path contain mgf file from DA(Bruker DataAnalysis) convert results
(filepathList, filenameList) = getfilepath(mgfFolder, ".mgf")
for filepath, filename in zip(filepathList, filenameList):
    print(f'Now read {filename}')
    myReader = mgfReader(filepath, saveFileName="./Search/tissue/kidney/DAExport", cpuNum=7, isBruker=True) # Parallel reading mgf files
    myReader.start()
    del myReader

Now read tissue-K-01_GA8_1_35018_5.3.236.mgf
Use 7 Process 
Total time 23.669898748397827
Save output in ./Search/kidney/DAExport/tissue-K-01_GA8_1_35018_5.3.236_DAExport.csv
Now read tissue-K-02_GB1_1_35019_5.3.236.mgf
Use 7 Process 
Total time 24.284887075424194
Save output in ./Search/kidney/DAExport/tissue-K-02_GB1_1_35019_5.3.236_DAExport.csv
Now read tissue-K-03_GB2_1_35020_5.3.236.mgf
Use 7 Process 
Total time 23.47310757637024
Save output in ./Search/kidney/DAExport/tissue-K-03_GB2_1_35020_5.3.236_DAExport.csv
Now read tissue-K-04_GB3_1_35021_5.3.236.mgf
Use 7 Process 
Total time 25.968851327896118
Save output in ./Search/kidney/DAExport/tissue-K-04_GB3_1_35021_5.3.236_DAExport.csv


In [3]:
#------------------------------------------------------
#MSDIAL txt and mgf CSV with the same name will be matched based on the m/z, RT and CCS
mgfReaderOutFolder = "./Search/tissue/kidney/DAExport" # input a folder path contain mgf readResult
txtFolder = "./Search/tissue/kidney" # input a folder path contain MSDIAL peak finding file.
(mgfExportFilePathList, mgfExportFileNameList) = getfilepath(mgfReaderOutFolder, '.csv')
(txtFilePathList, txtFileNameList) = getfilepath(txtFolder, '.txt')
print(1)
for mgf, txt, mgfName, txtName in zip(mgfExportFilePathList, txtFilePathList, mgfExportFileNameList, txtFileNameList):
    print(mgfName)
    print(txtName)
    myreader = Connector(txtFilePath=txt, mgfExportFilePath=mgf, cpuNum=7,savepath='./Search/kidney/MAP')
    myreader.multiProcessConvert()
    myreader.exportTxtWithMap()
    del myreader

1
tissue-K-01_GA8_1_35018_5.3.236_DAExport.csv
tissue-K-01_GA8_1_35018.txt
Use 7 Process 
Total time 32.730201721191406
Save Output in ./Search/kidney/MAP/tissue-K-01_GA8_1_35018_5.3.236_MAP.csv
tissue-K-02_GB1_1_35019_5.3.236_DAExport.csv
tissue-K-02_GB1_1_35019.txt
Use 7 Process 
Total time 32.98604464530945
Save Output in ./Search/kidney/MAP/tissue-K-02_GB1_1_35019_5.3.236_MAP.csv
tissue-K-03_GB2_1_35020_5.3.236_DAExport.csv
tissue-K-03_GB2_1_35020.txt
Use 7 Process 
Total time 32.825539112091064
Save Output in ./Search/kidney/MAP/tissue-K-03_GB2_1_35020_5.3.236_MAP.csv
tissue-K-04_GB3_1_35021_5.3.236_DAExport.csv
tissue-K-04_GB3_1_35021.txt
Use 7 Process 
Total time 32.52159309387207
Save Output in ./Search/kidney/MAP/tissue-K-04_GB3_1_35021_5.3.236_MAP.csv


In [2]:
#We need to merge MS-DIAL alignment output files into one.
nameDict = {'B': ['brain', 1], 'F': ['feces', 1], 'C': ['cardiac', 2], 'K': ['kidney', 3], 'G': ['gut', 2],
                'H': ['hepar', 3], 'L': ['lungs', 2]}
for key in nameDict.keys():
    print(key)
    txtMapFolder = f"./Search/tissue/{nameDict[key][0]}/MAP"
    mgfExportFolder = f"./Search/tissue/{nameDict[key][0]}/DAExport"
    alignmentFolder = f'./Search/tissue/{nameDict[key][0]}/alignment'
    filePathList, fileNameList = getfilepath(alignmentFolder,'.txt')
    areaID = None
    peakID = None
    for filePath in filePathList:
        if 'Area' in filePath:
            areaID = pd.read_csv(filePath,sep='\t')
        if 'PeakId' in filePath:
            peakID = pd.read_csv(filePath,sep='\t')
    peakID = peakID.iloc[:,[0,1,2,3,4,8,37,38,39,40]]
    areaID = areaID.iloc[:,[41,42]]
    areaID.rename(columns={'1':'Area_mean_','1.1':'Area_std_'},inplace=True)
    temp = pd.concat([peakID,areaID],axis=1)
    temp = temp.loc[temp.loc[:,'Average Mz'] <= 900]
    rtIndexId = list(temp.loc[temp.loc[:,'Average CCS'] == -1].index)
    i = 0
    temp[['Area Alignment ID','Area_mean','Area_std']] = None
    while i < len(rtIndexId)-1:
        temp.loc[rtIndexId[i]:(rtIndexId[i+1]-1)].loc[:, ['Area Alignment ID']] = temp.loc[rtIndexId[i],'Alignment ID']
        temp.loc[rtIndexId[i]:(rtIndexId[i+1]-1)].loc[:, ['Area_mean']] = round(temp.loc[rtIndexId[i], 'Area_mean_'],4)
        temp.loc[rtIndexId[i]:(rtIndexId[i+1]-1)].loc[:, ['Area_std']] = round(temp.loc[rtIndexId[i], 'Area_std_'],4)
        i += 1
    temp.loc[rtIndexId[i]:,:].loc[:, ['Area Alignment ID']] = temp.loc[rtIndexId[i], 'Alignment ID']
    temp.loc[rtIndexId[i]:,:].loc[:, ['Area_mean']] = temp.loc[rtIndexId[i], 'Area_mean_']
    temp.loc[rtIndexId[i]:,:].loc[:, ['Area_std']] = temp.loc[rtIndexId[i], 'Area_std_']
    print()
    temp.to_csv(f'./Search/tissue/{nameDict[key][0]}/alignment/{key}.txt',index=0,sep='\t')

B

F

C

K

G

H

L



In [5]:
# Search database for all samples
nameDict = {'B': ['brain', 1], 'F': ['feces', 1], 'C': ['cardiac', 2], 'K': ['kidney', 3], 'G': ['gut', 2],
                'H': ['hepar', 3], 'L': ['lungs', 2]} # number is after RT calibration database index;Feces = intestinal content
# nameDict = {'K': ['kidney', 3]}
txtMapFolder  = f"./Search/kidney\\MAP"
mgfExportFolder = f"./Search/kidney\\DAExport"
for key in nameDict.keys():
    txtMapFolder = f"./Search/tissue/{nameDict[key][0]}/MAP"
    mgfExportFolder = f"./Search/tissue/{nameDict[key][0]}/DAExport"
    alignmentFolder = f'./Search/tissue/{nameDict[key][0]}/alignment'
    searchData = searchMSDIALAlignExportWithMGF(msdialFilePath=txtMapFolder,
                                                alignmentPath=f'./Search/tissue/{nameDict[key][0]}/alignment/{key}.txt',
                                                sterolDatabase=f'./Search/database/STDatabaseQC{nameDict[key][1]}.csv',
                                                mgfExportPath=mgfExportFolder,
                                                msTolerance=25e-6, msmsTolerance=25e-6,
                                                piecewiseRTAscendingRange=[(0, 4), (0, 0)],
                                                piecewiseCCSAscendingRange=[(0, 1.5), (0, 0)],
                                                )
    searchData.match_sterol(searchWhich=searchData.standardSTDatabasePd, weight=[0, 0.5, 0.5], tolerance=0)
    searchData.saveResult(saveFolder='./Search',appendStr='standard')
    del searchData
    #
    searchData = searchMSDIALAlignExportWithMGF(msdialFilePath=txtMapFolder,
                                                alignmentPath=f'./Search/tissue/{nameDict[key][0]}\\alignment/{key}.txt',
                                                sterolDatabase=f'./Search/database/STDatabaseQC{nameDict[key][1]}.csv',
                                                mgfExportPath=mgfExportFolder,
                                                msTolerance=25e-6, msmsTolerance=25e-6,
                                                piecewiseRTAscendingRange=[(0, 0), (84*0.5,84*2)],
                                                piecewiseCCSAscendingRange=[(1.44*0.5, 1.44*2), (0, 0)],
                                                )
    searchData.match_sterol(searchWhich=searchData.predSTDatabasePd, weight=[0.4, 0.2, 0.4])
    searchData.saveResult(saveFolder='./Search',appendStr='predict')
    searchData.merge(standardResult=f"./Search/{key}.txt_standard.csv"
                    , predictResult=f"./Search/{key}.txt_predict.csv",
                    saveReuslt=f'./Search/{key}-result.csv') # Merge standard and predicted sterol identification results, retaining the top 3 sterols.

./Search\B.txt_standard.csv


./Search\B.txt_predict.csv


./Search\F.txt_standard.csv


./Search\F.txt_predict.csv


./Search\C.txt_standard.csv


...

./Search\G.txt_predict.csv


./Search\H.txt_standard.csv


./Search\H.txt_predict.csv


./Search\L.txt_standard.csv


./Search\L.txt_predict.csv
